In [1]:
import peewee as pw

In [7]:
class BaseModel(pw.Model):
    class Meta:
        db = pw.PostgresqlDatabase(
            'nlp_database', 
            user='postgres', 
            password='postgres', 
            host='0.0.0.0', 
            port=2345,
            autorollback=True,
        )
        db.connect()
        database = db
        
        
class User(BaseModel):
    username = pw.CharField(unique=True)
    email = pw.CharField(unique=True)
    password = pw.CharField()

User.create_table()

### analize the database

In [2]:
import peewee as pw
import pandas as pd
from src.database_logger.models import ModelPredictionV1, ModelMetricsV1
from src.database_logger.logger import DatabaseAnalaizerV1
db = DatabaseAnalaizerV1()


In [5]:
get_random_row = """
WITH random_row AS (
	SELECT * FROM model_prediction_v1 ORDER BY random() LIMIT 1
)
SELECT * FROM random_row;
"""

In [2]:
def get_random_sample_by_wandb_run_id(
	wandb_run_id: str,
	table_save_path: str = "./temp.csv",
	columns: list[str] = [
		'wandb_run_id',
		'model_name',
		'epoch',
		'context',
		'model_prediction',
		'actual_response',
	]
):
	random_row = db.execute_sql(get_random_row)
	random_row = random_row[0][1]
 
	data_1 = ModelPredictionV1.select().where(
		(ModelPredictionV1.prediction_id == random_row)
		&
		(ModelPredictionV1.wandb_run_id == wandb_run_id)
	).dicts()
 
	data_2 = pd.DataFrame.from_records(data_1)
	data_2[columns].to_csv(table_save_path, index=False)

In [76]:
get_random_sample_by_wandb_run_id(
	wandb_run_id="19rmj0yc",
	columns=[
		'wandb_run_id',
		'model_name',
		'epoch',
		'context',
		'model_prediction',
		'actual_response',
	]
)

In [3]:
def get_sample_with_metrics_by_wandb_run_id(
        wandb_run_id: str,
        table_save_path: str = "./temp.csv",
        columns: list[str] = [
            'wandb_run_id',
            'model_name',
            'prediction_id',
            'epoch',
            'context',
            'model_prediction',
            'actual_response',
            'valid_loss_epoch',
            'blue_score_epoch',
            'rougel_score_epoch',
            'charf_score_epoch',
        ],
        samples_amount: int = 1
    ):
    """
    similar to
    select * from public.model_prediction_v1 mp
    join model_metrics_v1 mmv on mp.epoch = mmv.epoch
    where 
    mp.prediction_id = '69_2' and 
    mp.wandb_run_id = '19rmj0yc' and 
    mmv.wandb_run_id = '19rmj0yc'
    """
    data = []
    for step in range(samples_amount):
        random_row = db.execute_sql(get_random_row)
        random_row = random_row[0][1]
    
        data_1 = ModelPredictionV1.select(ModelPredictionV1, ModelMetricsV1).join(
            ModelMetricsV1, pw.JOIN.INNER, on=(
            ModelMetricsV1.epoch == ModelPredictionV1.epoch
        )).where(
            (ModelMetricsV1.wandb_run_id == wandb_run_id)
            &
            (ModelPredictionV1.wandb_run_id == wandb_run_id)
            &
            (ModelPredictionV1.prediction_id == random_row)
        ).dicts()
    
        data_2 = pd.DataFrame.from_records(data_1)
        data.append(data_2)
    
    data = pd.concat(data)
    data[columns].to_csv(table_save_path, index=False)



In [6]:
get_sample_with_metrics_by_wandb_run_id(
	wandb_run_id="2e29af6q",
 	samples_amount=4
)